# TFX Pipeline
#### With Dummy Dataset

![header](assets/header-image.jpg)

The key benefit of machine learning pipelines lies in the automation of the model life cycle steps. When new training data becomes available, a workflow which includes data validation, preprocessing, model training, analysis, and deployment should be triggered.

Here's an image that shows the workflow of a common machine learning pipeline and also the corresponding tfx components to be used for its implementation.

![flow_and_components](assets/flow-and-components.png)

Installing required libraries

In [2]:
# !pip install -q -U tfx==1.4.0
# !pip install tensorflow==2.7.0
# !pip install tensorflow_decision_forests==0.2.1

In [59]:
# creating required directories
!mkdir -p saved_data
!mkdir -p model
!mkdir -p data

Adding all the imports

In [60]:
import tempfile
import tensorflow as tf
import urllib.request
import os
import pandas as pd
import shutil
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_decision_forests as tfdf

from absl import logging
from pathlib import Path
from tfx import v1 as tfx
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.proto import example_gen_pb2

print(f"Tensorflow Version: {tf.__version__}")
print(f"TFX Version: {tfx.__version__}")
print(f"TFDF Version: {tfdf.__version__}")

print(f"Tensorflow Data Validation Version: {tfdv.__version__}")

logging.set_verbosity(logging.INFO)

Tensorflow Version: 2.7.0
TFX Version: 1.4.0
TFDF Version: 0.2.1
Tensorflow Data Validation Version: 1.4.0


In [100]:
DATA_DIRECTORY = 'data'
DATA_SOURCE_PATH = Path(DATA_DIRECTORY) / 'Social_Network_Ads.csv'

SAVED_DATA = 'saved_data'
DATA_TRAIN_FILENAME = Path(SAVED_DATA) / 'train.csv'
DATA_TEST_FILENAME = Path(SAVED_DATA) / 'test.csv'

PIPELINE_NAME = 'sample-pipeline'
PIPELINE_DIRECTORY = os.path.join(Path('pipelines'), PIPELINE_NAME)

METADATA_PATH = Path("metadata") / PIPELINE_NAME / "metadata.db"

SCHEMA_DIRECTORY = os.path.join(PIPELINE_DIRECTORY, 'schema')
SCHEMA_FILE_NAME = str(os.path.join(SCHEMA_DIRECTORY, 'schema.pbtxt'))

MODEL_DIRECTORY = Path('model')

# Module Paths
CONSTANTS_MODULE_PATH = 'constants.py'
TRANSFORM_MODULE_PATH = 'transform.py'
TRAINER_MODULE_PATH = 'trainer.py'

In [62]:
data_df = pd.read_csv(DATA_SOURCE_PATH)

# splitting the data for training and testing
data_df = data_df.sample(frac=1)
train_df = data_df[: int(len(data_df) * 0.7)]
test_df = data_df[int(len(data_df) * 0.7): ]

# removing the undesired columns from all the datasets
datasets = [train_df, test_df]
drop_columns = ['User ID']
for dataset in datasets:
    dataset.drop(drop_columns, axis=1, inplace=True)

train_df.to_csv(DATA_TRAIN_FILENAME, index=False)
test_df.to_csv(DATA_TEST_FILENAME, index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [63]:
# peeking at the data
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 154 to 96
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Gender           280 non-null    object
 1   Age              280 non-null    int64 
 2   EstimatedSalary  280 non-null    int64 
 3   Purchased        280 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 10.9+ KB


## Common Functions

In [64]:
# creating the useful conversion functions

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(str(value), encoding="raw_unicode_escape")]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [90]:
# This function will be required to convert out test set 
# to compatible schema types for inferencing
def _examples(df):
    examples = []
    for index, row in df.iterrows():
        features = {
            "Gender": _bytes_feature(row['Gender']),
            "Age": _int64_feature(row['Age']),
            "EstimatedSalary": _int64_feature(row['EstimatedSalary']),
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=features))
        examples.append(example_proto.SerializeToString())

    return examples

#### Running components interactively

We are creating an interactive context that helps us in running each component in interactive mode.

In [66]:
from tfx.orchestration.experimental.interactive.interactive_context import (InteractiveContext)

context = InteractiveContext()

### Data Ingestion

The `ExampleGen` component can ingest a few data structures, including comma- separated value files (CSVs), precomputed TFRecord files, and serialization outputs from Apache Avro and Apache Parquet.

- The `CsvExampleGen` component from the tfx will be used to ingest the csv data.
- The train data will be split into train-eval set in the ration of 3:1.

In [67]:
output = tfx.proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     tfx.proto.SplitConfig.Split(name="train", hash_buckets=3),
                                                     tfx.proto.SplitConfig.Split(name="eval", hash_buckets=1)
    ]))
example_gen = tfx.components.CsvExampleGen(input_base=SAVED_DATA, output_config=output)

context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data saved_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:2,total_bytes:6956,xor_checksum:0,sum_checksum:3276503522"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Generating Statistics

The `StatisticsGen` component will be used to generate the statistics of the features present in the data.  

These statistics help us in understanding the characteristics of the data. It recieves the example outputs generated by the previous generator component and generates the stats for each splits of the data.

In [68]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


### Generating Data Schema

Data schema are a form of describing the representation of your data‐ sets. The schema definition of your dataset can then be used to validate future datasets to determine if they are in line with your previous training sets. 

The easiest way to get the schema for the dataset is to infer it using the `SchemaGen` component.

In [69]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'Gender',STRING,required,,'Gender'
'Age',INT,required,,-
'EstimatedSalary',INT,required,,-
'Purchased',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"


#### Updating the schema

We have inferred the types and features in the dataset.  
But we can make the required changes in the schema as well.   

The two environments that we need to create are:
- `TRAINING`: This environment will be used to validate the data for the training process.  
- `SERVING`: This environment will be used to validate the data for the model in production.  

We will remove the `Purchased` (target feature) from the serving environment.

In [70]:
schema = tfdv.load_schema_text(
    os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, "schema.pbtxt")
)

# adding the needed environments
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# removing the `Purchased` column from the serving environment
tfdv.get_feature(schema, "Purchased").not_in_environment.append("SERVING")

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Gender',STRING,required,,'Gender'
'Age',INT,required,,-
'EstimatedSalary',INT,required,,-
'Purchased',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"


The importer node will be used to load a saved schema into the pipeline for subsequent components.

In [71]:
!mkdir -p {SCHEMA_DIRECTORY}

tfdv.write_schema_text(schema, SCHEMA_FILE_NAME)

# loading the updated schema using the importer node.
schema_importer = tfx.dsl.Importer(
    source_uri=str(SCHEMA_DIRECTORY),
    artifact_type=tfx.types.standard_artifacts.Schema
).with_id("schema_importer")

context.run(schema_importer)
context.show(schema_importer.outputs['result'])

INFO:absl:Running driver for schema_importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: pipelines/sample-pipeline/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for schema_importer
INFO:absl:Running publisher for schema_importer
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'Gender',STRING,required,,'Gender'
'Age',INT,required,,-
'EstimatedSalary',INT,required,,-
'Purchased',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"


###  Data Validation

The data validation step checks that the data in your pipelines is what your feature engineering step expects. It assists you in comparing multiple datasets. It also highlights if your data changes over time, for example, if your training data is significantly different from the new data provided to your model for inference.

Data validation performs three distinct checks:  
    • Check for data anomalies.  
    • Check that the data schema hasn’t changed.  
    • Check that the statistics of our new datasets still align with statistics from our previous training datasets.  

In [72]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']
)

context.run(example_validator)
context.show(example_validator.outputs['anomalies'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/ExampleValidator/anomalies/5/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/ExampleValidator/anomalies/5/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


### Data Transformation

Data Transformation can be used to standardize some steps in the machine learning pipeline.  
- Preprocessing your data efficiently in the context of the entire dataset
- Scaling the preprocessing steps effectively
- Avoiding a potential training-serving skew

The `transform` component will be used here which makes sure that the same pre-processing is done on the training as well as the serving data. 

We need to implement a `preprocessing_fn()` in order to do the necessary transformations.

In [74]:
%%writefile {CONSTANTS_MODULE_PATH}
LABEL = 'Purchased'

Overwriting constants.py


In [76]:
%%writefile {TRANSFORM_MODULE_PATH}

import tensorflow as tf
import tensorflow_transform as tft

import constants

LABEL = constants.LABEL

def preprocessing_fn(inputs):

    outputs = dict()
    outputs['Age'] = inputs['Age']
    outputs['EstimatedSalary'] = inputs['EstimatedSalary']
    # converting the `Gender` into label encoded column.
    outputs['Gender'] = tf.cast(tf.equal(inputs['Gender'], 'male'), tf.int64)

    outputs[LABEL] = inputs[LABEL]

    return outputs

Overwriting transform.py


Creating the transform component.

In [77]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=os.path.abspath(TRANSFORM_MODULE_PATH),
)
context.run(transform, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/content/transform.py' (including modules: ['trainer', 'constants', 'transform']).
INFO:absl:User module package has hash fingerprint version e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmp5ldyz37q/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp8_j5wbbs', '--dist-dir', '/tmp/tmp2fp39iek']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/_wheels/tfx_user_code_Transform-0.0+e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/_wheels/tfx_user_code_Transform-0.0+e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB conne

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Transform/transform_graph/6/.temp_path/tftransform_tmp/e74cd2d11c8e4573bc37ec74c517a6c1/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Transform/transform_graph/6/.temp_path/tftransform_tmp/e74cd2d11c8e4573bc37ec74c517a6c1/assets


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 23
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 24
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 9
        type_id: 18
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Transform/pre_transform_schema/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"


We can take a look at the transformed examples from our data.

In [78]:
train_uri = os.path.join(
    transform.outputs['transformed_examples'].get()[0].uri,
    'Split-train'
)

tfrecord_filenames = [
                      os.path.join(train_uri, name) for name in os.listdir(train_uri)
]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    print(example)

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "EstimatedSalary"
    value {
      int64_list {
        value: 54000
      }
    }
  }
  feature {
    key: "Gender"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Purchased"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 35
      }
    }
  }
  feature {
    key: "EstimatedSalary"
    value {
      int64_list {
        value: 108000
      }
    }
  }
  feature {
    key: "Gender"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Purchased"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 34
      }
    }
  }
  feature {
    key: "EstimatedSalary"
    value {
      int64_list {


### Model Training

To train the model we need to create a `trainer` component. 

The Trainer component will look for a `run_fn()` function in our module file and use the function as an entry point to execute the training process.  

The transformations needed to be included as part of the default signature of the model.

In [80]:
%%writefile {TRAINER_MODULE_PATH}

import tensorflow as tf
import tensorflow_decision_forests as tfdf
import tensorflow_transform as tft

from absl import logging
from tensorflow.keras import layers, Model, optimizers, losses, metrics
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from typing import List, Text

import constants

LABEL = constants.LABEL
BATCH_SIZE = 32
EPOCHS = 50

def _input_fn(
    file_pattern: List[Text],
    data_accessor: tfx.components.DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    batch_size: int,
) -> tf.data.Dataset:
    """
    Generates a dataset of features that can be used to train
    and evaluate the model.

    Args:
        file_pattern: List of paths or patterns of input data files.
        data_accessor: An instance of DataAccessor that we can use to
            convert the input to a RecordBatch.
        tf_transform_output: The transformation output.
        batch_size: The number of consecutive elements that we should
            combine in a single batch.

    Returns:
        A dataset that contains a tuple of (features, indices) where 
            features is a dictionary of Tensors, and indices is a single
            Tensor of label indices.
    """

    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
        schema=tf_transform_output.raw_metadata.schema,
    )

    tft_layer = tf_transform_output.transform_features_layer()

    def apply_transform(raw_features):
        transformed_features = tft_layer(raw_features)
        transformed_label = transformed_features.pop(LABEL)
        return transformed_features, transformed_label
    
    return dataset.map(apply_transform).repeat()

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """
    Returns a function that parses a serialized tf.Example and applies
    the transformations during inference.
    Args:
        model: The model that we are serving.
        tf_transform_output: The transformation output that we want to 
            include with the model.
    """
    
    # setting up the model's tft_layer
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        
        # we have to ensure that the target column is not in required list of columns.
        required_feature_spec = {
            k: v for k, v in feature_spec.items() if k != LABEL
        }

        parsed_features = tf.io.parse_example(
            serialized_tf_examples,
            required_feature_spec
        )

        transformed_features = model.tft_layer(parsed_features)

        # running the inference with the transformed features.
        return model(transformed_features)

    return serve_tf_examples_fn

def _model() -> tf.keras.Model:
    """
    To create a Keras model.
    """

    inputs = [
              layers.Input(shape=(1,), name="Age"),
              layers.Input(shape=(1,), name="EstimatedSalary"),
              layers.Input(shape=(1,), name="Gender")
    ]

    x = layers.concatenate(inputs)
    x = layers.Dense(8, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=optimizers.Adam(1e-2),
        loss="binary_crossentropy",
        metrics=[metrics.BinaryAccuracy()],
    )

    model.summary(print_fn=logging.info)
    return model

def run_fn(fn_args: tfx.components.FnArgs):
    """
    The callback function that will be called by the Trainer component
    to train the model using the suplied arguments.

    Args:
        fn_args: A collection of name/value pairs representing the 
            arguments to train the model.
    """
    
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=BATCH_SIZE,
    )

    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=BATCH_SIZE,
    )

    model = _model()
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        epochs=EPOCHS
    )

    # We need to modify the default signature to include the transform layer in 
    # the computational graph.
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(model, tf_transform_output),
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

Overwriting trainer.py


We can now add a trainer component to train a model.

In [81]:
trainer = tfx.components.Trainer(
    examples=example_gen.outputs["examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=5),
    module_file=os.path.abspath(TRAINER_MODULE_PATH),
)
context.run(trainer, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/content/trainer.py' (including modules: ['trainer', 'constants', 'transform']).
INFO:absl:User module package has hash fingerprint version e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmprhfjfj2x/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmplt0gdefy', '--dist-dir', '/tmp/tmpxdlcub3_']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/_wheels/tfx_user_code_Trainer-0.0+e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/_wheels/tfx_user_code_Trainer-0.0+e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initia

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Feature Gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature EstimatedSalary has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Purchased has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Model: "model_2"
INFO:absl:__________________________________________________________________________________________________
INFO:absl: Layer (type)                   Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl: Age (InputLayer)               [(None, 1)]          0           []                               
INFO:absl:                                                                                                  
INFO:absl: EstimatedSalary (InputLayer)   [(Non

Epoch 1/50
100/100 [==============================] - 1s 6ms/step - loss: 267.8823 - binary_accuracy: 0.5066 - val_loss: 106.0862 - val_binary_accuracy: 0.7000
Epoch 2/50
100/100 [==============================] - 0s 4ms/step - loss: 83.4841 - binary_accuracy: 0.5178 - val_loss: 73.6792 - val_binary_accuracy: 0.3000
Epoch 3/50
100/100 [==============================] - 0s 5ms/step - loss: 119.2248 - binary_accuracy: 0.4991 - val_loss: 143.3806 - val_binary_accuracy: 0.7250
Epoch 4/50
100/100 [==============================] - 0s 5ms/step - loss: 110.0067 - binary_accuracy: 0.5119 - val_loss: 570.8691 - val_binary_accuracy: 0.3000
Epoch 5/50
100/100 [==============================] - 0s 5ms/step - loss: 152.6463 - binary_accuracy: 0.4953 - val_loss: 71.6391 - val_binary_accuracy: 0.2875
Epoch 6/50
100/100 [==============================] - 0s 5ms/step - loss: 93.2976 - binary_accuracy: 0.5069 - val_loss: 49.7022 - val_binary_accuracy: 0.7000
Epoch 7/50
100/100 [=========================

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Trainer/model/7/Format-Serving/assets
INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Trainer/model/7/Format-Serving. ModelRun written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Trainer/model_run/7
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 26
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 27
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Model Evaluation

Since we are done with creating and training a model, we need to evaluate its performance before we can deploy it.  
The performance can be checked using the `Evaluator` component.  

We are going to look at the `val_accuracy` of the model in this example, from two perspectives:
- For the overall dataset.
- For the individual genders to ensure that the model is fair to both.

In [83]:
eval_config = tfma.EvalConfig(
    model_specs=[
                 tfma.ModelSpec(
                     signature_name="serving_default",
                     preprocessing_function_names=['tft_layer'],
                     label_key="Purchased",
                 )
    ],
    metrics_specs = [
                     tfma.MetricsSpec(
                         per_slice_thresholds={
                             "binary_accuracy": tfma.PerSliceMetricThresholds(
                                 thresholds=[
                                             tfma.PerSliceMetricThreshold(
                                                 slicing_specs=[tfma.SlicingSpec()],
                                                 threshold=tfma.MetricThreshold(
                                                     value_threshold=tfma.GenericValueThreshold(
                                                         lower_bound={"value":0.7}
                                                     ),
                                                     change_threshold=tfma.GenericChangeThreshold(
                                                         direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                                         absolute={"value": -1e-10},
                                                     ),
                                                 ),
                                             )
                                 ]
                             ),
                         }
                     )
    ],
    slicing_specs=[
                   tfma.SlicingSpec(),
                   tfma.SlicingSpec(feature_keys=["Gender"])
    ]
)

We want to deploy only when our model is outperforming the previous models and that model is termed as *blessed model*.  

`Resolver` node is used to compare the current model with the previous one and deploy the blessed one.

In [84]:
model_resolver = tfx.dsl.Resolver(
    strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
    model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessings=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
).with_id("latest_blessed_model_resolver")

context.run(model_resolver)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessings: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [85]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    eval_config=eval_config,
    baseline_model=model_resolver.outputs["model"],
)

context.run(evaluator, enable_cache=False)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "per_slice_thresholds": {\n        "binary_accuracy": {\n          "thresholds": [\n            {\n              "slicing_specs": [\n                {}\n              ],\n              "threshold": {\n                "change_threshold": {\n                  "absolute": -1e-10,\n                  "direction": "HIGHER_IS_BETTER"\n                },\n                "value_threshold": {\n                  "lower_bound": 0.7\n                }\n              }\n            }\n          ]\n        }\n      }\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Purchased",\n      "preprocessing_function_names": [\n        "tft_layer"\n      ],\n      "signature_name": "serving_default"\n    }\n  ],\n 


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d5a653150> and <keras.engine.input_layer.InputLayer object at 0x7f3d5ce51410>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d5a653150> and <keras.engine.input_layer.InputLayer object at 0x7f3d5ce51410>).
INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "per_slice_thresholds": {\n        "binary_accuracy": {\n          "thresholds": [\n            {\n              "slicing_specs": [\n                {}\n              ],\n              "threshold": {\n                "change_threshold": {\n                  "absolute": -1e-10,\n                  "direction": "HIGHER_IS_BETTER"\n                },\n                "value_threshold": {\n                  "lower_bound": 0.7\n                }\n              }\n            }\n          ]\n        }\n      }\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Purchased",\n      "


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d5cc6dc90> and <keras.engine.input_layer.InputLayer object at 0x7f3d5cc5f650>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d5cc6dc90> and <keras.engine.input_layer.InputLayer object at 0x7f3d5cc5f650>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3e50ce0450> and <keras.engine.input_layer.InputLayer object at 0x7f3dd071fd10>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3e50ce0450> and <keras.engine.input_layer.InputLayer object at 0x7f3dd071fd10>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d58a1bd10> and <keras.engine.input_layer.InputLayer object at 0x7f3d58a2eb50>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d58a1bd10> and <keras.engine.input_layer.InputLayer object at 0x7f3d58a2eb50>).
INFO:absl:Evaluation complete. Results written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Evaluator/evaluation/9.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Evaluator/blessing/9.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 16
        type_id: 30
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Evaluator/evaluation/9"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 30
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 31
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Evaluator/blessing/9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Trainer/model/7"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Pushing the model

The model will be pushed if the model is blessed.

In [86]:
pusher = tfx.components.Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=str(MODEL_DIRECTORY)
        )
    ),
)

context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1638251875
INFO:absl:Model written to serving path model/1638251875.
INFO:absl:Model pushed to /tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Pusher/pushed_model/10.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 18
        type_id: 33
        uri: "/tmp/tfx-interactive-2021-11-30T05_56_03.974969-yzfsa_v5/Pusher/pushed_model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "model/1638251875"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1638251875"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 33
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Inferencing

Now we can run the inferences on the pushed model.

In [87]:
def get_inference_fn(model_directory):
    model_directories = (d for d in os.scandir(model_directory) if d.is_dir())
    model_path = max(model_directories, key=lambda i: int(i.name)).path

    loaded_model = tf.keras.models.load_model(model_path)

    return loaded_model.signatures["serving_default"]

In [91]:
inference_fn = get_inference_fn(MODEL_DIRECTORY)
result = inference_fn(examples=tf.constant(_examples(test_df)))
print(result["output_0"].numpy())


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d589a8790> and <keras.engine.input_layer.InputLayer object at 0x7f3dd0451110>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d589a8790> and <keras.engine.input_layer.InputLayer object at 0x7f3dd0451110>).


[[0.03220685]
 [0.46029472]
 [0.5184892 ]
 [0.17297481]
 [0.03745157]
 [0.67159355]
 [0.05825204]
 [0.08253836]
 [0.03109209]
 [0.14211378]
 [0.01026638]
 [0.54865664]
 [0.00307088]
 [0.11654628]
 [0.0051414 ]
 [0.04682363]
 [0.11298493]
 [0.6830877 ]
 [0.02954738]
 [0.19606875]
 [0.04218721]
 [0.00264895]
 [0.26473102]
 [0.05261479]
 [0.00801955]
 [0.84577835]
 [0.09096617]
 [0.00268115]
 [0.00590247]
 [0.14293896]
 [0.00727324]
 [0.00475701]
 [0.14112698]
 [0.19416368]
 [0.11682624]
 [0.0055347 ]
 [0.08142502]
 [0.02018276]
 [0.04202535]
 [0.04229586]
 [0.4267416 ]
 [0.07690777]
 [0.01121056]
 [0.02657227]
 [0.00833632]
 [0.03555671]
 [0.23300353]
 [0.00363223]
 [0.03032882]
 [0.1238111 ]
 [0.01567248]
 [0.01212719]
 [0.16708386]
 [0.01242075]
 [0.20534608]
 [0.04540218]
 [0.23566048]
 [0.08876477]
 [0.14211378]
 [0.13409239]
 [0.46330968]
 [0.00233451]
 [0.53326845]
 [0.04616746]
 [0.08898594]
 [0.02742235]
 [0.00609537]
 [0.20777936]
 [0.08233638]
 [0.02595322]
 [0.10678811]
 [0.03

## Pipeline

We can define the schema manually and run the entire pipeline.

In [94]:
%%writefile {SCHEMA_FILE_NAME}

feature {
  name: "Gender"
  type: BYTES
  domain: "Gender"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Age"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "EstimatedSalary"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Purchased"
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "SERVING"
  shape {
    dim {
      size: 1
    }
  }
}
string_domain {
  name: "Gender"
  value: "Female"
  value: "Male"
}
default_environment: "TRAINING"
default_environment: "SERVING"

Overwriting pipelines/sample-pipeline/schema/schema.pbtxt


### Running the pipeline

In [97]:
import tensorflow_model_analysis as tfma

def create_pipeline(
    pipeline_name: str,
    pipeline_directory: str,
    data_directory: str,
    schema_path: str,
    model_directory: str,
    metadata_path: str,
    transform_module_path: str,
    trainer_module_path: str,
    )-> tfx.dsl.Pipeline:

    output = tfx.proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     tfx.proto.SplitConfig.Split(name="train", hash_buckets=3),
                                                     tfx.proto.SplitConfig.Split(name="eval", hash_buckets=1)
    ]))
    example_gen = tfx.components.CsvExampleGen(input_base=SAVED_DATA, output_config=output)

    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples']
    )

    schema_importer = tfx.dsl.Importer(
        source_uri=str(SCHEMA_DIRECTORY),
        artifact_type=tfx.types.standard_artifacts.Schema
    ).with_id("schema_importer")

    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_importer.outputs['result']
    )

    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_importer.outputs['result'],
        module_file=os.path.abspath(TRANSFORM_MODULE_PATH),
    )

    trainer = tfx.components.Trainer(
        examples=example_gen.outputs["examples"],
        transform_graph=transform.outputs["transform_graph"],
        train_args=tfx.proto.TrainArgs(num_steps=100),
        eval_args=tfx.proto.EvalArgs(num_steps=5),
        module_file=os.path.abspath(TRAINER_MODULE_PATH),
    )

    eval_config = tfma.EvalConfig(
        model_specs=[
                    tfma.ModelSpec(
                        signature_name="serving_default",
                        preprocessing_function_names=['tft_layer'],
                        label_key="Purchased",
                    )
        ],
        metrics_specs = [
                        tfma.MetricsSpec(
                            per_slice_thresholds={
                                "binary_accuracy": tfma.PerSliceMetricThresholds(
                                    thresholds=[
                                                tfma.PerSliceMetricThreshold(
                                                    slicing_specs=[tfma.SlicingSpec()],
                                                    threshold=tfma.MetricThreshold(
                                                        value_threshold=tfma.GenericValueThreshold(
                                                            lower_bound={"value":0.7}
                                                        ),
                                                        change_threshold=tfma.GenericChangeThreshold(
                                                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                                            absolute={"value": -1e-10},
                                                        ),
                                                    ),
                                                )
                                    ]
                                ),
                            }
                        )
        ],
        slicing_specs=[
                    tfma.SlicingSpec(),
                    tfma.SlicingSpec(feature_keys=["Gender"])
        ]
    )

    model_resolver = tfx.dsl.Resolver(
        strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
        model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
        model_blessings=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
    ).with_id("latest_blessed_model_resolver")

    evaluator = tfx.components.Evaluator(
        examples=example_gen.outputs["examples"],
        model=trainer.outputs["model"],
        eval_config=eval_config,
        baseline_model=model_resolver.outputs["model"],
    )

    pusher = tfx.components.Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=tfx.proto.PushDestination(
            filesystem=tfx.proto.PushDestination.Filesystem(
                base_directory=str(MODEL_DIRECTORY)
            )
        ),
    )

    components = [
                  example_gen,
                  statistics_gen,
                  schema_importer,
                  example_validator,
                  transform,
                  trainer,
                  model_resolver,
                  evaluator,
                  pusher
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_directory,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(
            metadata_path
            ),
            components=components,
    )

In [103]:
tfx.orchestration.LocalDagRunner().run(
    create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_directory=str(PIPELINE_DIRECTORY),
        data_directory=DATA_DIRECTORY,
        schema_path=str(SCHEMA_DIRECTORY),
        model_directory=str(MODEL_DIRECTORY),
        metadata_path=str(METADATA_PATH),
        transform_module_path=TRANSFORM_MODULE_PATH,
        trainer_module_path=TRAINER_MODULE_PATH
    )
)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/transform.py' (including modules: ['trainer', 'constants', 'transform']).
INFO:absl:User module package has hash fingerprint version e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpq1110qux/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpz2s0jkly', '--dist-dir', '/tmp/tmpliy4sev0']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/sample-pipeline/_wheels/tfx_user_code_Transform-0.0+e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@pipelines/sample-pipeline/_wheels/tfx_user_code_Transform-0.0+e743a58cd10fd86e3c3a1181d29fc5e40f1ecc0d04cf50e7103fce5e259a0af4-py3-none-any.whl'
INFO:abs

INFO:tensorflow:Assets written to: pipelines/sample-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/449ca0cbece04589a94073520e23c8a8/assets


INFO:tensorflow:Assets written to: pipelines/sample-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/449ca0cbece04589a94073520e23c8a8/assets


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d57673b90> and <keras.engine.input_layer.InputLayer object at 0x7f3d5766f250>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d573e0f90> and <keras.engine.input_layer.InputLayer object at 0x7f3d57426590>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d573e0f90> and <keras.engine.input_layer.InputLayer object at 0x7f3d57426590>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d56192250> and <keras.engine.input_layer.InputLayer object at 0x7f3d561922d0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d56192250> and <keras.engine.input_layer.InputLayer object at 0x7f3d561922d0>).
INFO:absl:Evaluation complete. Results written to pipelines/sample-pipeline/Evaluator/evaluation/8.
INFO:absl:Checking validation results.
INFO:absl:Blessing result False written to pipelines/sample-pipeline/Evaluator/blessing/8.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 8 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'blessing': [Artifact(artifact: uri: "pipelines/sample-pipeline/Evaluator/blessing/8"
custom_properties {
  key: "name"
  value {
    string_value: "sample-pipeline:2021-11-30T06:15:53.934297:Evaluator:blessing:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "ModelBlessing"
)], 'e

### Running the inferencing

In [105]:
inference_fn = get_inference_fn(MODEL_DIRECTORY)
result = inference_fn(examples=tf.constant(_examples(test_df)))
print(result["output_0"].numpy())


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d556c1690> and <keras.engine.input_layer.InputLayer object at 0x7f3d556bb3d0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3d556c1690> and <keras.engine.input_layer.InputLayer object at 0x7f3d556bb3d0>).


[[0.03220685]
 [0.46029472]
 [0.5184892 ]
 [0.17297481]
 [0.03745157]
 [0.67159355]
 [0.05825204]
 [0.08253836]
 [0.03109209]
 [0.14211378]
 [0.01026638]
 [0.54865664]
 [0.00307088]
 [0.11654628]
 [0.0051414 ]
 [0.04682363]
 [0.11298493]
 [0.6830877 ]
 [0.02954738]
 [0.19606875]
 [0.04218721]
 [0.00264895]
 [0.26473102]
 [0.05261479]
 [0.00801955]
 [0.84577835]
 [0.09096617]
 [0.00268115]
 [0.00590247]
 [0.14293896]
 [0.00727324]
 [0.00475701]
 [0.14112698]
 [0.19416368]
 [0.11682624]
 [0.0055347 ]
 [0.08142502]
 [0.02018276]
 [0.04202535]
 [0.04229586]
 [0.4267416 ]
 [0.07690777]
 [0.01121056]
 [0.02657227]
 [0.00833632]
 [0.03555671]
 [0.23300353]
 [0.00363223]
 [0.03032882]
 [0.1238111 ]
 [0.01567248]
 [0.01212719]
 [0.16708386]
 [0.01242075]
 [0.20534608]
 [0.04540218]
 [0.23566048]
 [0.08876477]
 [0.14211378]
 [0.13409239]
 [0.46330968]
 [0.00233451]
 [0.53326845]
 [0.04616746]
 [0.08898594]
 [0.02742235]
 [0.00609537]
 [0.20777936]
 [0.08233638]
 [0.02595322]
 [0.10678811]
 [0.03

## References

- [Tensorflow Extended Documentation](https://www.tensorflow.org/tfx/)
- [Building Machine Learning Pipelines Book](https://www.amazon.in/Building-Machine-Learning-Pipelines-Automating/dp/9385889001/ref=sr_1_1?keywords=building+machine+learning+pipelines&qid=1638266556&sr=8-1)
- [TFX Pipelines by Santiago](https://deepnote.com/@svpino/TFX-Pipelines-Titanic-Dataset-4btM0tK3TNydUMOTgR4dbQ)
